In [24]:
!reentry scan -r aiida
!verdi daemon restart 

Restarting the daemon... OK


In [25]:
import aiida
from aiida.orm import Code
aiida.load_profile()
code = Code.get_from_string('siesta-psml-lua@pwk')

In [26]:
#-----------------------------------------------------------------------------------------
# Structure Parts
#-----------------------------------------------------------------------------------------
#print(code)
from aiida.orm import StructureData
cell = [[15.0, 00.0 , 00.0,],
        [00.0, 15.0 , 00.0,],
        [00.0, 00.0 , 15.0,],
        ]
s = StructureData(cell=cell)
s.append_atom(position=( 0.000,  0.000,  0.000 ),symbols=['O']) #1
s.append_atom(position=( 0.757,  0.586,  0.000 ),symbols=['H']) #2
s.append_atom(position=(-0.757,  0.586,  0.000 ),symbols=['H']) #3 
s.append_atom(position=( 0.000,  3.500,  0.000 ),symbols=['O']) #4
s.append_atom(position=( 0.757,  2.914,  0.000 ),symbols=['H']) #5
s.append_atom(position=(-0.757,  2.914,  0.000 ),symbols=['H']) #6
# If want to save the CIF
#cif_file=s.get_cif()
#cif_file.export('/home/aakhtar/calculations/siesta/test-neb-aiida/with-remote-folder/structure','cif')

In [27]:
#-----------------------------------------------------------------------------------------
# Settings Parts
#-----------------------------------------------------------------------------------------
from aiida.orm import Dict
parameters = Dict(dict={
   "mesh-cutoff": "250 Ry",
   "dm-tolerance": "0.0001",
   "MD-TypeOfRun":   "LUA",
   "LUA-script":   "neb_with_restart-new.lua",
   "DM-NumberPulay ":  "3",
   "DM-History-Depth":  "0",
   "SCF-Mixer-weight":  "0.02",
   "SCF-Mix":   "density",
   "SCF-Mixer-kick":  "35",
   "MD-VariableCell":  "F",
   "MD-MaxCGDispl":  "0.3 Bohr",
   "MD-MaxForceTol":  " 0.04000 eV/Ang",
    "%block Geometry-Constraints":
    """
    atom [1 -- 4]
    %endblock Geometry-Constraints"""    
    })
#parameters.get_dict()
#parent_calc_folder

In [28]:
#-----------------------------------------------------------------------------------------
# Pseudo Parts
#-----------------------------------------------------------------------------------------
import os
#from aiida_siesta.data.psml import PsmlData
from aiida_siesta.data.psf import PsfData
pseudo_file_to_species_map = [ ("H.psf", ['H']),("O.psf", ['O'])]
pseudos_dict = {}
for fname, kinds, in pseudo_file_to_species_map:
      absname = os.path.realpath(os.path.join("./",fname))
      pseudo, created = PsfData.get_or_create(absname, use_first=True)
      for j in kinds:
              pseudos_dict[j]=pseudo
pseudos_dict

{'H': <PsfData: uuid: a170b9e5-9c3e-4a0c-abd8-40a320e6ed04 (pk: 35)>,
 'O': <PsfData: uuid: 9b3d6b90-a0a6-4f9b-9dcc-fca3422a6e0e (pk: 36)>}

In [29]:
#-----------------------------------------------------------------------------------------
# LUA Parts
#-----------------------------------------------------------------------------------------
from aiida_siesta.data.lua import LUAData
lua_file_to_species_map = [ ("neb_with_restart-new.lua", ['lua']),
                            ("image_0.xyz", ['images0']),
                            ("image_1.xyz", ['images1']),
                            ("image_2.xyz", ['images2']),
                            ("image_3.xyz", ['images3']),
                            ("image_4.xyz", ['images4']),
                            ("image_5.xyz", ['images5']),
                            ("image_6.xyz", ['images6']),
                          ]
lua_dict = {}
for fname, kinds, in lua_file_to_species_map:
      absname = os.path.realpath(os.path.join("./",fname))
      lua, created = LUAData.get_or_create(absname, use_first=True)
      for j in kinds:
              lua_dict[j]=lua
lua_dict

{'lua': <LUAData: uuid: 63d9d210-2a26-4270-a594-5ae78966f019 (pk: 37)>,
 'images0': <LUAData: uuid: bd0793fb-d4bb-48f6-a713-4c788b6d6ae8 (pk: 38)>,
 'images1': <LUAData: uuid: dd0975df-867c-460f-8bc9-033f4ec92224 (pk: 66)>,
 'images2': <LUAData: uuid: 40192b86-c1de-420c-adf1-387728542e2c (pk: 67)>,
 'images3': <LUAData: uuid: c30cd2f1-887a-4f85-afb1-e0281bd61387 (pk: 68)>,
 'images4': <LUAData: uuid: d303f045-e8fb-41c9-9f19-cb635490f0b9 (pk: 69)>,
 'images5': <LUAData: uuid: a528b264-2903-4b42-92a2-7e46db2bbc0a (pk: 70)>,
 'images6': <LUAData: uuid: 674cac9f-0fa7-44c6-9016-928321bdae6f (pk: 44)>}

In [30]:
#-----------------------------------------------------------------------------------------
# Remote Folder for DM.1... Parts
# Have to set correct path for remote folder
#-----------------------------------------------------------------------------------------
from aiida.orm import RemoteData ,load_computer,load_node
a=RemoteData()
a.computer=load_computer('1')
a.set_remote_path("/home/aakhtar/Calculations/aiida/neb/test/RemoteFolder/")
a.store()
folder=load_node(a.pk)

In [31]:
#-----------------------------------------------------------------------------------------
# Basis and kpoint Parts
#-----------------------------------------------------------------------------------------
from aiida.orm import Dict
basis_dict = {
  "%block PAO-Basis":
    """
 O                     2                    # Species label, number of l-shells
 n=2   0   2                         # n, l, Nzeta
   3.305      2.510
   1.000      1.000
 n=2   1   2 P   1                   # n, l, Nzeta, Polarization, NzetaPol
   3.937      2.542
   1.000      1.000
H                     1                    # Species label, number of l-shells
 n=1   0   2 P   1                   # n, l, Nzeta, Polarization, NzetaPol
   4.828      3.855
   1.000      1.000

    %endblock PAO-Basis""",
}

basis = Dict(dict=basis_dict)
from aiida.orm import KpointsData
kpoints=KpointsData()
kp_mesh = 1
mesh_displ = 0.5 #optional
#kpoints.set_kpoints_mesh([kp_mesh,kp_mesh,kp_mesh],[mesh_displ,mesh_displ,mesh_displ])
kpoints.set_kpoints_mesh([1,1,1],[mesh_displ,mesh_displ,mesh_displ])

In [32]:
#-----------------------------------------------------------------------------------------
# Retrieved Parts
#-----------------------------------------------------------------------------------------
settings_dict = {
  'additional_retrieve_list': ['aiida.EIG', 'aiida.ORB_INDX','NEB.results','NEB.1.T'],
}
settings = Dict(dict=settings_dict)

In [33]:
#-----------------------------------------------------------------------------------------
# Basis and kpoint Parts
#-----------------------------------------------------------------------------------------
from aiida_siesta.calculations.siesta import SiestaCalculation
builder = SiestaCalculation.get_builder()
builder.code = code
builder.structure = s
builder.parameters = parameters
builder.pseudos = pseudos_dict
builder.basis = basis
builder.kpoints = kpoints
builder.parent_calc_folder =folder
builder.settings=settings
builder.luafiles = lua_dict
builder.metadata.options.resources = {'num_machines': 1,
                                     "num_mpiprocs_per_machine": 6}
builder.metadata.options.withmpi= True
builder.metadata.options.max_wallclock_seconds = 864000
print(builder)

{'metadata': {'options': {'resources': {'num_machines': 1, 'num_mpiprocs_per_machine': 6}, 'withmpi': True, 'max_wallclock_seconds': 864000}}, 'pseudos': {'H': <PsfData: uuid: a170b9e5-9c3e-4a0c-abd8-40a320e6ed04 (pk: 35)>, 'O': <PsfData: uuid: 9b3d6b90-a0a6-4f9b-9dcc-fca3422a6e0e (pk: 36)>}, 'luafiles': {'lua': <LUAData: uuid: 63d9d210-2a26-4270-a594-5ae78966f019 (pk: 37)>, 'images0': <LUAData: uuid: bd0793fb-d4bb-48f6-a713-4c788b6d6ae8 (pk: 38)>, 'images1': <LUAData: uuid: dd0975df-867c-460f-8bc9-033f4ec92224 (pk: 66)>, 'images2': <LUAData: uuid: 40192b86-c1de-420c-adf1-387728542e2c (pk: 67)>, 'images3': <LUAData: uuid: c30cd2f1-887a-4f85-afb1-e0281bd61387 (pk: 68)>, 'images4': <LUAData: uuid: d303f045-e8fb-41c9-9f19-cb635490f0b9 (pk: 69)>, 'images5': <LUAData: uuid: a528b264-2903-4b42-92a2-7e46db2bbc0a (pk: 70)>, 'images6': <LUAData: uuid: 674cac9f-0fa7-44c6-9016-928321bdae6f (pk: 44)>}, 'code': <Code: Remote code 'siesta-psml-lua' on pwk, pk: 26, uuid: 348b1f10-53dc-40ee-8f70-ebde5

In [35]:
#-----------------------------------------------------------------------------------------
# running Parts
#-----------------------------------------------------------------------------------------
from aiida.engine import run,submit #,submit_test
results = run(builder)
#results=submit_test(builder)

In [36]:
#results
results['forces_and_stress'].pk

86

In [69]:
from aiida.orm import load_node
neb_results_node=load_node(82) #load_node('584')
neb_resutls_retrieved=neb_results_node.outputs.retrieved 
neb_results=neb_resutls_retrieved.get_object_content('NEB.results')

In [70]:
import numpy as np
Number_of_images=5
data_temp=[]
data_temp.append(neb_results.splitlines())
data=[]
for line in range(len(data_temp[0])):
        if line>2:
            #print (line)
            data.append(data_temp[0][line])
del data_temp
while '' in data:   #removing Null strings between each neb stage
    data.remove('')
image_number=[]
reaction_coordinates=[]
Energy=[]
E_diff=[]
Curvature=[]
Max_Force=[]
for i in range(len(data)):
    image_number.append(float(data[i].split()[0]))
    reaction_coordinates.append(float(data[i].split()[1]))
    Energy.append(float(data[i].split()[2]))
    E_diff.append(float(data[i].split()[3]))
    Curvature.append(float(data[i].split()[4]))
    Max_Force.append(float(data[i].split()[5]))
#Last NEB
Total_Number=Number_of_images+2
shift=len(E_diff)-Total_Number
#-----------------------------------------------
# Final Values
#-----------------------------------------------
image_number_Final=np.array([])
reaction_coordinates_Final=np.array([])
Energy_Final=np.array([])
E_diff_Final=np.array([])
Curvature_Final=np.array([])
Max_Force_Final=np.array([])
for i in range(Total_Number):
    image_number_Final=np.append(image_number_Final,int(image_number[shift+i]))
    reaction_coordinates_Final=np.append(reaction_coordinates_Final,reaction_coordinates[shift+i])
    E_diff_Final=np.append(E_diff_Final,E_diff[shift+i])
    Energy_Final=np.append(Energy_Final,Energy[shift+i])
    Curvature_Final=np.append(Curvature_Final,Curvature[shift+i])
    Max_Force_Final=np.append(Max_Force_Final,Max_Force[shift+i])
min_neb=max(Energy_Final)
max_neb=min(Energy_Final)
Barrier=abs(max_neb-min_neb)

In [72]:
E_diff_Final

array([-0.11815006, -0.03935191, -0.04646751, -0.01624707, -0.03635964,
       -0.09785523, -0.11815006])

In [ ]:
#---------------------
#Examples
#------------------------
#b=load_node('567')
#b.get_arraynames()
#b.get_array('forces')
#---------------------------------
#Example for sending DM or images
#---------------------------------
#q=QueryBuilder
#q.append(computer)
#for i in q.iterall():
#    print (i)
#from aiida.orn import RemoteData
#a=RemoteData()
#a.computer=load_computer(2)
#a.set_remote_path("")
#a.store()

In [ ]:
#folder=load_node('')
#builer.parnet_calc_folder=folder